In [0]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import tqdm

In [2]:
# transform=transforms.Compose([transforms.RandomHorizontalFlip(p=0.5),
#                               transforms.RandomVerticalFlip(p=0.5),
#                               transforms.ToTensor(),
#                                transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
transform=transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
trainset=torchvision.datasets.CIFAR10(root='./data', train=True,
                                    download=True, transform=transform)
trainloader=torch.utils.data.DataLoader(trainset, batch_size=5,
                                          shuffle=True, num_workers=0)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=1,
                                         shuffle=False, num_workers=0)

classes=('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, (3,3))
        self.batch1=nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 32, (3,3))
        self.batch2=nn.BatchNorm2d(32)
        self.drop1=nn.Dropout2d(p=0.2)
        self.conv3=nn.Conv2d(32,64,(3,3))
        self.batch3=nn.BatchNorm2d(64)
        self.conv4=nn.Conv2d(64,64,(3,3))
        self.batch4=nn.BatchNorm2d(64)
        self.drop2=nn.Dropout2d(p=0.3)
        self.conv5=nn.Conv2d(64,128,(3,3))
        self.batch5=nn.BatchNorm2d(128)
        self.conv6=nn.Conv2d(128,128,(3,3))
        self.batch6=nn.BatchNorm2d(128)
        self.drop3=nn.Dropout2d(p=0.4)
        self.fc1=nn.Linear(128*(4*4), 10)
        self.pool=nn.MaxPool2d((2,2), 2)
        self.pad=nn.ZeroPad2d(1)

    # def adjust_learning_rate(optimizer, epoch):
    #     if epoch==8:
    #         lr = args.lr/10
    #     lr = args.lr * (0.1 ** (epoch // 30))
    #     for param_group in optimizer.param_groups:
    #         param_group['lr'] = lr

    def forward(self, x):
        x = self.batch1(F.elu(self.conv1(self.pad(x))))
        x = self.batch2(F.elu(self.conv2(self.pad(x))))
        x = self.drop1(self.pool(x))
        x = self.batch3(F.elu(self.conv3(self.pad(x))))
        x = self.batch4(F.elu(self.conv4(self.pad(x))))
        x = self.drop2(self.pool(x))
        x = self.batch5(F.elu(self.conv5(self.pad(x))))
        x = self.batch6(F.elu(self.conv6(self.pad(x))))
        x = self.drop3(self.pool(x))
        x = x.view(-1, 128*(4*4))
        x = self.fc1(x)
        return x

net = Net()

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
net.to(device)
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adagrad(net.parameters())
scheduler=optim.lr_scheduler.MultiStepLR(optimizer, milestones=[8,30,50,70], gamma=0.3)

cuda:0


In [0]:
for epoch in range(80): # no of epochs
    
    running_loss=0.0
    
    net.train()
    
    for i,data in enumerate(tqdm.notebook.tqdm(trainloader)):
          inputs, labels = data[0].to(device), data[1].to(device)
          optimizer.zero_grad()
          outputs=net(inputs)
          loss=criterion(outputs,labels)
          loss.backward()
          optimizer.step()
          running_loss+=loss.item()
          # if i%2000==1999:
          #       if i%12000==11999: running_loss=0
          #       #   print('Epoch:',epoch+1,', loss:',running_loss/2000)

    correct=0
    total=0
    net.eval()
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs=net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    scheduler.step()
    print("Epoch=",epoch+1,", Accuracy=",correct*100/total, " Epoch loss=",running_loss)
print('Model trained :)')


Epoch= 1 , Accuracy= 60.92  Epoch loss= 16666.35076096654



Epoch= 2 , Accuracy= 68.83  Epoch loss= 11552.000289730728



Epoch= 3 , Accuracy= 71.8  Epoch loss= 9996.421788860112



Epoch= 4 , Accuracy= 74.3  Epoch loss= 9187.50147328712



Epoch= 5 , Accuracy= 75.31  Epoch loss= 8571.79579312168


...


Epoch= 10 , Accuracy= 78.83  Epoch loss= 6720.099424757063



Epoch= 11 , Accuracy= 78.96  Epoch loss= 6604.8285511163995



Epoch= 12 , Accuracy= 79.35  Epoch loss= 6556.093177631497



Epoch= 13 , Accuracy= 79.47  Epoch loss= 6474.4131835363805



Epoch= 14 , Accuracy= 79.51  Epoch loss= 6422.808461518493


In [0]:
# torch.save(net.state_dict(), './cifar2.pth')